### Summary

In order to add Yelp data to our raw data, we thought it would be most efficient to build functions through which we would pass each row of raw data. We adapted code samples from Yelp's GitHub, in order to first identify the yelp record for a restaurant based on name and address. Once the Yelp ID# was acquired, we used it to pull the ratings information. The results were stored in a Pandas DataFrame, merged together, and saved to a new csv file.

In [1]:
#import necessary dependencies
import pandas as pd
import requests
import json
from pprint import pprint
from urllib.error import HTTPError
from urllib.parse import quote
from urllib.parse import urlencode
import sys
import time

In [2]:
#determine global variables
api_key = "YOUR KEY HERE"
api_host = 'https://api.yelp.com'
match_path = '/v3/businesses/matches'
detail_path = '/v3/businesses/'
detail_list = []

In [3]:
#use a function to pull the yelp ids
def request(host, path, api_key, url_params=None):
    """Given your API_KEY, send a GET request to the API.
    Args:
        host (str): The domain host of the API.
        path (str): The path of the API after the domain.
        API_KEY (str): Your API Key.
        url_params (dict): An optional set of query parameters in the request.
    Returns:
        dict: The JSON response from the request.
    Raises:
        HTTPError: An error occurs from the HTTP request.
    """
    url_params = url_params or {}
    url = '{0}{1}'.format(host, quote(path.encode('utf8')))
    headers = {
        'Authorization': 'Bearer %s' % api_key,
    }

    response = requests.request('GET', url, headers=headers, params=url_params)

    return response.json()

In [4]:
#use a function to get the business details
def get_business(api_key, business_id):
    """Query the Business API by a business ID.
    Args:
        business_id (str): The ID of the business to query.
    Returns:
        dict: The JSON response from the request.
    """
    business_path = detail_path + business_id

    return request(api_host, business_path, api_key)

In [5]:
#import the csv you plan to search with
#MAKE SURE TO CHANGE THIS TO THE CSV YOU NEED TO READ
#file name to use: parsed/#la_restaurants.csv (replace # with the file number you are calling)

r_target = pd.read_csv('YOUR FILE HERE')


In [6]:
total_calls = int(r_target['program_name'].count())
print(total_calls)

2497


In [7]:
#iterate over the DataFrame to call the API for each business in the csv
for i in r_target.index:
    
    try:
    
        #display which pull the loop is on in order to track progress
        pull = i+1
        completed = round((pull/total_calls)*100,1)
        call_count = len(detail_list)
        sys.stdout.write(f"\rCalling {pull} of {total_calls} -- {completed}% complete. {call_count} successful calls made!")


        #pull the search criteria for the API
        name = r_target.loc[i,'program_name']
        add1 = r_target.loc[i,'facility_address']
        city = r_target.loc[i,'facility_city']
        state = "CA"
        country = "US"
        url_params = {'name': name,
                     'address1': add1,
                     'city': city,
                     'state': state,
                     'country': country}

        #call the api to find the business
        get_id = request(api_host, match_path, api_key, url_params)

        #extract the yelp id for the business, if there is one
        try:
            if len(get_id['businesses']) > 0:
                y_id = get_id['businesses'][0]['id']
            else:
                y_id = "none"
        except Exception as e: 
                y_id = "key_error"


        #if the api found the business, call the api to pull the necessary details
        #use the restaurant address as a unqiue identifier so the resulting DataFrame can be merged
        if y_id != "none" and y_id !="key_error":
            detail = get_business(api_key, y_id)
            if 'price' in detail:
                price = detail['price']
            else:
                price = "blank"
            if 'rating' in detail:
                rating = detail['rating']
            else:
                rating = "blank"
            if 'review_count' in detail:
                review_count = detail['review_count']
            else:
                review_count = 'blank'

            r_details = {'facility_address': add1,
                        'yelp_id': y_id,
                        'price': price,
                        'rating': rating,
                        'review_count': review_count,
                        'food_type': [d['title'] for d in detail['categories']],
                        'latitude': detail['coordinates']['latitude'],
                        'longitude': detail['coordinates']['longitude'],
                        'in_business': detail['is_closed']}
            detail_list.append(r_details)

            #clear the previous status update so the new one may be displayed
            sys.stdout.flush()
    except:
        time.sleep(60)
        print("Connection issue. Will resume in 1 minute")
        continue

Calling 601 of 2497 -- 24.1% complete. 499 successful calls made!Connection issue. Will resume in 1 minute
Calling 833 of 2497 -- 33.4% complete. 690 successful calls made!Connection issue. Will resume in 1 minute
Calling 1588 of 2497 -- 63.6% complete. 1354 successful calls made!Connection issue. Will resume in 1 minute
Calling 1684 of 2497 -- 67.4% complete. 1430 successful calls made!Connection issue. Will resume in 1 minute
Calling 2353 of 2497 -- 94.2% complete. 2007 successful calls made!Connection issue. Will resume in 1 minute
Calling 2500 of 2497 -- 100.1% complete. 2132 successful calls made!

In [8]:
#check to see how many successful calls were made
calls = len(detail_list)
print(f"{calls} successful call(s) were made out of {total_calls} attempts")

2133 successful call(s) were made out of 2497 attempts


In [9]:
#store the yelp id query results in a dataframe
details_pd = pd.DataFrame(detail_list)
details_pd.head()

,facility_address,food_type,in_business,latitude,longitude,price,rating,review_count,yelp_id
0,1356 ALLISON AVE,[Izakaya],False,34.072670,-118.251780,$$$,4.5,141,Aq1vM8LCcb8mdpCOKCdlsw
1,6374 W SUNSET BLVD unit B,[Conveyor Belt Sushi],False,34.097723,-118.328453,$$,3.5,437,9nOfyJebgtj-NmaACdoYgg
2,535 S GRAND AVE,"[Bars, American (Traditional)]",False,34.049392,-118.254873,$$,3.5,127,VI8FM3Jhtwn0XsmVJAD-Tg
3,11678 SAN VICENTE BLVD,"[Vegan, Vegetarian, Health Markets]",True,34.053632,-118.464851,$$,3.5,95,IXH9nn4ooDOjcltBdyru8w
4,901 S VERMONT AVE,"[Barbeque, Korean]",False,34.055786,-118.291992,$$,3.5,603,cfmt0lJ0Jkt9cBq3NoRRNw


In [10]:
#merge the DataFrames
yelp_scrub = r_target.merge(details_pd, on='facility_address')

In [11]:
#for the love of all things good, save your calls to a csv!
#MAKE SURE TO CHANGE THE NUMBER ON THE END TO THE CORRESPONDING NUMBER OF THE CSV YOU INITIALLY READ
#file name to use: "called/la_restaurants_yelped#.csv" (replace # with the file number you are calling)
yelp_scrub.to_csv("YOUR FILE HERE")